# Data Structures

You have some collection of data: how can you store and manipulate it?

Python provides a wide range of options, from the simple and flexible to the sophisticated and massively scaleable.

## Contents

Works on your laptop:
- [Core Python](#core)
- [Python collections](#collections)
- [Numpy arrays](#numpy)
- [Pandas](#pandas)
- [Astropy Tables](#tables)
- [Apache Arrow](#arrow)

Works with big data:
- [Vaex](#vaex)
- [Dask](#dask)
- [HDF5](#hdf5)
- [Apache Spark](#spark)
- [Relational databases & SQL](#sql)

<a id='core'></a>  

## Core Python

All these have been built into Python since its early days and can be used without imports. These should be pretty familiar to anyone who has used Python and are just included here for completeness.

### Lists and tuples
Ordered sequences with zero-based implicit index. Lists are mutable, tuples are immutable but otherwise similar.

### Dicts
What other languages call dictionaries, hash tables or associative arrays.. A set of key:value pairs, where the key must be unique and immutable (hashable types: string, int, float, tuple; __not__ list). The value can be pretty much anything, including nested lists, dicts, etc.

### Sets
An __unordered__ collection of unique values. Duplicates are silently removed.

<a id='collections'></a>  

## Python collections

### namedtuple
Allow positions within a tuple to be referred to by a meaningful name, not just integer index as with regular tuples.

### deque
Double-ended queue. Entries can be efficiently appended or popped at either end.

### ChainMap
Groups multiple dicts or other mappings together to create a single, updateable view.

### Counter
A specialized dictionary class for working with counts (of hashable keys). The value is restricted to integers, but there are useful extra methods to operate on these.

### OrderedDict
Preserve the order in which items were added and the iterator return them in the same order. Standard dicts are much more random.

### defaultdict
Much like a standard dict, except it can create missing keys as needed instead of throwing a KeyError.

### UserDict, UserList, UserString
Wrappers useful as base classes for programmers to write their own extensions.

In [1]:
from collections import deque, OrderedDict, namedtuple, defaultdict
d = OrderedDict.fromkeys('abcde')
d

OrderedDict([('a', None), ('b', None), ('c', None), ('d', None), ('e', None)])

<a id='numpy'></a>  

## NumPy

### Arrays
Python lists are wonderfully flexible. However, flexibility is the enemy of performance optimization, and lists don't scale well. Numpy adds an array type with these features:

- they are created with a specified size and a precisely specified datatype, so memory can be allocated at creation time.
- ONLY the specified datatype can be stored in each array (they are homogeneous and the data type is immutable). 
- arrays can be N-dimensional (and very often are in scientific work)
- they can be passed to numpy ufuncs as arguments, processing the whole array much more efficiently than by writing a loop.

### Structured arrays
An extension of ndarrays to contain a sequence of named fields. Maybe not something most programmers often use directly, but several other data structures rely on them internally. Details: https://docs.scipy.org/doc/numpy/user/basics.rec.html

This example pulls in some SDSS data in structured array format. Note the dtype definitions: (name, format) tuples for each column. Columns can be referred to by name as a readable alternative to the index number.

In [3]:
import numpy as np
data = np.load('sdss_colors.npy')
display(data[:3])
data['redshift'][:5]

array([(18.06726, 17.92133, 17.76799, 17.83525, 17.74435, 'QSO', 0.9460756 , 2.688471e-04),
       (19.46527, 17.84728, 17.00071, 16.56581, 16.20629, 'GALAXY', 0.07831088, 2.122711e-05),
       (18.65488, 17.34237, 16.79778, 16.49659, 16.24572, 'GALAXY', 0.0329398 , 1.859143e-05)],
      dtype=[('u', '<f8'), ('g', '<f8'), ('r', '<f8'), ('i', '<f8'), ('z', '<f8'), ('specClass', '<U6'), ('redshift', '<f8'), ('redshift_err', '<f8')])

array([0.9460756 , 0.07831088, 0.0329398 , 0.04208763, 0.05497772])

<a id='pandas'></a>  

## Pandas

A big topic! Pandas evolve the idea of structured arrays into dataframes and add a set of methods for manipulating them. Which soulds simple, but the methods aren't just pretty-printing and some statistics. They seem to include (or at least aspire to) most of what is in the R language plus much of what is in SQL.

Pandas are widely used and quite popular with most people, not least because they are the basis for other packages such as Seaborn (plotting) and scikit-learn (machine learning). Oddly, the most outspoken critic is perhaps the man who created pandas: http://wesmckinney.com/blog/apache-arrow-pandas-internals/

In [4]:
import pandas as pd

Import data, calculate some new columns:

In [5]:
df = pd.DataFrame(np.load('sdss_colors.npy'))
df['u-g'] = df['u'] - df['g']
df['g-r'] = df['g'] - df['r']
df['r-i'] = df['r'] - df['i']
display(df.shape)
df.head()

(1000, 11)

,u,g,r,i,z,specClass,redshift,redshift_err,u-g,g-r,r-i
0,18.06726,17.92133,17.76799,17.83525,17.74435,QSO,0.946076,0.000269,0.14593,0.15334,-0.06726
1,19.46527,17.84728,17.00071,16.56581,16.20629,GALAXY,0.078311,0.000021,1.61799,0.84657,0.43490
2,18.65488,17.34237,16.79778,16.49659,16.24572,GALAXY,0.032940,0.000019,1.31251,0.54459,0.30119
3,18.51099,16.54340,15.63836,15.20576,14.85911,GALAXY,0.042088,0.000013,1.96759,0.90504,0.43260
4,18.91729,17.03743,16.13777,15.71413,15.39045,GALAXY,0.054978,0.000013,1.87986,0.89966,0.42364


Summary statistics:

In [6]:
df.describe()

,u,g,r,i,z,redshift,redshift_err,u-g,g-r,r-i
count,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000
mean,18.827523,17.561124,16.944155,16.612269,16.383019,0.250575,0.011150,1.266399,0.616969,0.331885
std,0.702003,0.911264,1.091257,1.183789,1.263518,0.500840,0.350824,0.576579,0.295969,0.166345
min,14.483200,12.856720,11.826220,11.363090,10.954490,0.000000,0.000000,-0.733830,-0.548550,-2.151960
25%,18.537633,17.104602,16.290995,15.857525,15.542277,0.056498,0.000009,1.049523,0.432048,0.284950
50%,19.016785,17.624225,16.934760,16.571195,16.312055,0.083965,0.000014,1.300920,0.643930,0.376450
75%,19.332918,18.085977,17.578115,17.264513,17.081527,0.129447,0.000022,1.759927,0.860165,0.428692
max,19.599840,19.775030,20.188540,21.079170,21.043890,5.652918,11.094080,2.417470,1.576920,0.638780


Filter:

In [7]:
galaxies = df[data['specClass'] == 'GALAXY']
qsos = df[data['specClass'] == 'QSO']
display(galaxies.shape, qsos.shape)

(837, 11)

(163, 11)

Grouping and aggregation (of redshifts in this case):

In [8]:
df.groupby('specClass')['redshift'].aggregate([min, np.mean, np.median, max])

,min,mean,median,max
specClass,,,,
GALAXY,0.000000,0.079874,0.077432,0.489077
QSO,0.047992,1.127116,1.042593,5.652918


<a id='tables'></a>  

## Astropy Tables

These are something fairly similar to pandas but they are a separate thing developed in parallel by astronomers rather than data scientists. They have better support for astropy units and FITS files (naturally), also multidimensional columns. 

Plotting works well with Matplotlib, not with Seaborn or anything else that expects a pandas dataframe.

In [9]:
from astropy.table import Table
tbl = Table(np.load('sdss_colors.npy'))
tbl[:3]

u,g,r,i,z,specClass,redshift,redshift_err
float64,float64,float64,float64,float64,str6,float64,float64
18.06726,17.92133,17.76799,17.83525,17.74435,QSO,0.9460756,0.0002688471
19.46527,17.84728,17.00071,16.56581,16.20629,GALAXY,0.07831088,2.122711e-05
18.65488,17.34237,16.79778,16.49659,16.24572,GALAXY,0.0329398,1.859143e-05


In [10]:
tbl.info

<Table length=1000>
    name      dtype 
------------ -------
           u float64
           g float64
           r float64
           i float64
           z float64
   specClass    str6
    redshift float64
redshift_err float64

In [11]:
tbl['redshift'].format = '7.3f'
tbl.show_in_notebook(display_length=10)

idx,u,g,r,i,z,specClass,redshift,redshift_err
0,18.06726,17.92133,17.76799,17.83525,17.74435,QSO,0.946,0.0002688471
1,19.46527,17.84728,17.00071,16.56581,16.20629,GALAXY,0.078,2.122711e-05
2,18.65488,17.34237,16.79778,16.49659,16.24572,GALAXY,0.033,1.859143e-05
3,18.51099,16.5434,15.63836,15.20576,14.85911,GALAXY,0.042,1.305551e-05
4,18.91729,17.03743,16.13777,15.71413,15.39045,GALAXY,0.055,1.253144e-05
5,19.14975,17.96591,17.1856,16.73291,16.44635,GALAXY,0.154,1.135913e-05
6,18.79822,18.67259,18.47921,18.52675,18.4364,QSO,0.949,0.0002673743
7,18.86939,16.88241,15.819,15.28006,14.84171,GALAXY,0.102,1.819819e-05
8,19.24538,18.71721,18.32339,18.24627,18.08278,QSO,0.911,0.0001437659
9,19.53371,18.75589,18.2605,17.93995,17.8565,GALAXY,0.157,5.526448e-06


There are no units with this table, so let's create another data table for planet-like bodies. Astropy tables can either associate a unit with a column (which is backwards compatible), or store quantity objects directly (preferred for new code). A QTable is just a Table that defaults to Quantity objects.

In [12]:
from astropy import units as u
from astropy.table import QTable

Read in a csv file with Solar System objects, and don't involve me in any arguments about which should be called a "planet".

In [14]:
bodies_tbl = Table.read('planets_etc.csv', format='ascii')
bodies_tbl

Planet,a_AU,P_yr,v_km_s,e,i_deg,rot_period_h,eq_incl_deg,M_kg,R_eq_km,density,rings,T_mean_K
str7,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,str3,int64
Mercury,0.3871,0.2408,47.9,0.206,7.0,1407.6,0.0,3.3e+23,2439.5,5.43,No,440
Venus,0.7233,0.6152,35.0,0.007,3.39,-5832.5,177.3,4.87e+24,6052.0,5.24,No,737
Earth,1.0,1.0,29.8,0.017,0.0,23.9,23.4,5.97e+24,6378.0,5.52,No,288
Mars,1.5273,1.8809,24.1,0.093,1.85,24.6,25.2,6.42e+23,3397.0,3.04,No,208
Ceres,2.76596,4.599,17.882,0.07976,10.587,9.0,3.0,9e+21,473.0,2.16,No,168
Jupiter,5.2028,11.862,13.1,0.048,1.31,9.9,3.1,1.9e+27,71490.0,1.33,Yes,163
Saturn,9.5388,29.458,9.6,0.056,2.49,10.7,26.7,5.68e+26,560270.0,0.7,Yes,133
Uranus,19.1914,84.01,6.8,0.046,0.77,-17.2,97.9,8.68e+25,25560.0,1.3,Yes,78
Neptune,30.0611,164.79,5.4,0.01,1.77,16.1,29.6,1.02e+26,24765.0,1.76,Yes,73


Get the units out of the heading and into the table cells:

In [15]:
bodies_qtbl = QTable()
bodies_qtbl['name'] = bodies_tbl['Planet']
bodies_qtbl['a'] = bodies_tbl['a_AU']*u.AU
bodies_qtbl['P'] = bodies_tbl['P_yr']*u.year
bodies_qtbl['v'] = bodies_tbl['v_km_s']*u.km/u.s
bodies_qtbl['e'] = bodies_tbl['e']
bodies_qtbl['i'] = bodies_tbl['i_deg']*u.deg
bodies_qtbl['rot_period'] = bodies_tbl['rot_period_h']*u.h
bodies_qtbl['eq_incl'] = bodies_tbl['eq_incl_deg']*u.deg
bodies_qtbl['M'] = bodies_tbl['M_kg']*u.kg
bodies_qtbl['R_eq'] = bodies_tbl['R_eq_km']*u.km
bodies_qtbl['rho'] = bodies_tbl['density']*u.g/u.cm**3
bodies_qtbl['rings'] = (bodies_tbl['rings'] == 'Yes')
bodies_qtbl['T_mean'] = bodies_tbl['T_mean_K']*u.K
bodies_qtbl

name,a,P,v,e,i,rot_period,eq_incl,M,R_eq,rho,rings,T_mean
,AU,yr,km / s,,deg,h,deg,kg,km,g / cm3,,K
str7,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,bool,float64
Mercury,0.3871,0.2408,47.9,0.206,7.0,1407.6,0.0,3.3e+23,2439.5,5.43,False,440.0
Venus,0.7233,0.6152,35.0,0.007,3.39,-5832.5,177.3,4.87e+24,6052.0,5.24,False,737.0
Earth,1.0,1.0,29.8,0.017,0.0,23.9,23.4,5.97e+24,6378.0,5.52,False,288.0
Mars,1.5273,1.8809,24.1,0.093,1.85,24.6,25.2,6.42e+23,3397.0,3.04,False,208.0
Ceres,2.76596,4.599,17.882,0.07976,10.587,9.0,3.0,9e+21,473.0,2.16,False,168.0
Jupiter,5.2028,11.862,13.1,0.048,1.31,9.9,3.1,1.9e+27,71490.0,1.33,True,163.0
Saturn,9.5388,29.458,9.6,0.056,2.49,10.7,26.7,5.68e+26,560270.0,0.7,True,133.0
Uranus,19.1914,84.01,6.8,0.046,0.77,-17.2,97.9,8.68e+25,25560.0,1.3,True,78.0


The columns with dimensions are astropy Quantities. The units are displayed, they can be converted, and calculations on columns understand the units.

In [16]:
display(bodies_qtbl[1]['name'], bodies_qtbl[1]['a'], bodies_qtbl[1]['a'].to(u.km))

'Venus'

<Quantity 0.7233 AU>

<Quantity 1.0820414e+08 km>

In [17]:
from astropy.constants import GM_sun

# Keplerian orbit calculation
bodies_qtbl['P_calc'] = np.sqrt(4*np.pi**2*bodies_qtbl['a']**3/GM_sun).to(u.year)
bodies_qtbl['P'] - bodies_qtbl['P_calc'] # difference should be near zero

<Quantity [-4.79528742e-05,  4.30996063e-05, -1.88867462e-05,
           -6.63376828e-03, -1.20351221e-03, -5.62732039e-03,
           -3.09041169e-03, -6.52540729e-02, -3.21233790e-02,
            4.43976414e-03] yr>

QTables can be written to and read from FITS files, preserving the units:

In [18]:
bodies_qtbl.write('bodies.fits', format='fits')

tmp = QTable.read('bodies.fits')
tmp

name,a,P,v,e,i,rot_period,eq_incl,M,R_eq,rho,rings,T_mean,P_calc
,AU,yr,km / s,,deg,h,deg,kg,km,g / cm3,,K,yr
bytes7,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,bool,float64,float64
Mercury,0.3871,0.2408,47.9,0.206,7.0,1407.6,0.0,3.3e+23,2439.5,5.43,False,440.0,0.24084795287424313
Venus,0.7233,0.6152,35.0,0.007,3.39,-5832.5,177.3,4.87e+24,6052.0,5.24,False,737.0,0.6151569003936662
Earth,1.0,1.0,29.8,0.017,0.0,23.9,23.4,5.97e+24,6378.0,5.52,False,288.0,1.0000188867461788
Mars,1.5273,1.8809,24.1,0.093,1.85,24.6,25.2,6.42e+23,3397.0,3.04,False,208.0,1.8875337682776234
Ceres,2.76596,4.599,17.882,0.07976,10.587,9.0,3.0,9e+21,473.0,2.16,False,168.0,4.600203512208458
Jupiter,5.2028,11.862,13.1,0.048,1.31,9.9,3.1,1.9e+27,71490.0,1.33,True,163.0,11.86762732039321
Saturn,9.5388,29.458,9.6,0.056,2.49,10.7,26.7,5.68e+26,560270.0,0.7,True,133.0,29.461090411691707
Uranus,19.1914,84.01,6.8,0.046,0.77,-17.2,97.9,8.68e+25,25560.0,1.3,True,78.0,84.07525407294416


<a id='arrow'></a>  

## Apache Arrow

Officially: "Apache Arrow is a cross-language development platform for in-memory data. It specifies a standardized language-independent columnar memory format for flat and hierarchical data, organized for efficient analytic operations on modern hardware."

The idea is to define a common in-memory data format to support the needs of many platforms, avoiding duplication of effort, and minimizing difficult format conversions when sharing data. A priority is therefore to integrate efficiantly with everything: pandas, Spark, vaex, Impala, etc

# Big Data

A phrase that has become wildly overused, in this context it means data that certainly won't fit in RAM, may not fit on your disk drive, and may be distributed across multiple servers. Numpy arrays and pandas weren't designed for this and certainly won't cope.

Common features of software designed for big data include:
- memory mapping, so only portions of a file are read in as needed
- highly parallel scatter-gather algorithms which avoid memory copy
- lazy evaluation: the system stores details of how to perform a calculation, rather than immediately carrying it out and storing the result.

Inevitably, this is an area of active research and the packages tend to be relatively immature. Evolving fast, though: there's a lot of money at stake in getting this right.

<a id='vaex'></a>  

## Vaex

Aims to have much of the feel of pandas, but with lazy out-of-core dataframes that scale much better to billion-plus rows. My first impressions are that most things work as expected and this is an easy package to get started with.

Installation is simple: `conda install vaex`. Documentation : https://vaex.readthedocs.io/en/latest/

In [19]:
import vaex
dfv = vaex.example() # example data, from a server in the Netherlands

In [20]:
dfv

#,E,FeH,L,Lz,random_index,vx,vy,vz,x,y,z
0,-121238.171875,-2.309227609164518,831.0799560546875,-336.426513671875,1511648,53.276722,288.386047,-95.2649078,-0.777470767,2.10626292,1.93743467
1,-100819.9140625,-1.788735491591229,1435.1839599609375,-828.7567749023438,2728665,252.810791,-69.9498444,-56.3121033,3.77427316,2.23387194,3.76209331
2,-100559.9609375,-0.7618109022478798,1039.2989501953125,920.802490234375,1202632,96.276474,226.440201,-34.7527161,1.3757627,-6.3283844,2.63250017
3,-70174.8515625,-1.5208778422936413,2441.724853515625,1183.5899658203125,1020502,204.968842,-205.679016,-58.9777031,-7.06737804,1.31737781,-6.10543537
4,-144138.75,-2.655341358427361,374.8164367675781,-314.5353088378906,3154816,-311.742371,-238.41217,186.824127,0.243441463,-0.822781682,-0.206593871
...,...,...,...,...,...,...,...,...,...,...,...
"329,995",-119687.3203125,-1.6499842518381402,746.8833618164062,-508.96484375,1919483,107.432999,-2.13771296,17.5130272,3.76883793,4.66251659,-4.42904139
"329,996",-68933.8046875,-1.4336036247720836,2395.633056640625,1275.490234375,1064141,32.0,108.089264,179.060638,9.17409325,-8.87091351,-8.61707687
"329,997",-112580.359375,-1.9306227597361942,1182.436279296875,115.58557891845703,374845,8.46711349,-38.2765236,-127.541473,-1.14041007,-8.4957695,2.25749826
"329,998",-74862.90625,-1.225019818838568,1324.5926513671875,1057.017333984375,425745,110.221558,-31.3925591,86.2726822,-14.2985935,-5.51750422,-8.65472317


Operations involving multiple rows and summary statistics are very fast. We have 400,000 rows here, $10^{10}$ rows are said to be viable.

In [21]:
dfv.mean(dfv.vx / dfv.vy), dfv.std(dfv.vx / dfv.vy)

(-1.750682689701575, 4000.7209707492702)

'Virtual columns' are stored as expressions (how to calculate) not numbers (results of calculating):

In [22]:
type(dfv.vx / dfv.vy)

vaex.expression.Expression

I/O is pretty flexible. Unlike pandas, vaex is astronomer-friendly with `from_astropy_table()` and `export_fits()` methods.

<a id='dask'></a>  

## Dask

A library for parallel computing in Python. Dask is installed by default in Anaconda. It simulates numpy arrays and pandas dataframes in a more scaleable way, while remaining fairly lightweight compared with Spark.

In [23]:
import dask.dataframe as dd

___TODO___ - add this section when I get my head round it

<a id='hdf5'></a>  

## HDF5

A file format, but also a way of structuring hierarchical data. There are two different Python interfaces: h5py and PyTables. See the `files - HDF5` notebook for more details.

<a id='spark'></a>  

## Apache Spark

This is getting to the big stuff: a distributed cluster-computing framework for handling massive datasets. Through PySpark, a Jupyter notebook can be a very good Spark client, but it needs server software running on each node of the cluster to do the heavy lifting. So a bit hard to demonstrate here.

The central concept is a Resilient Distributed Database, RDD, which you can use much like a pandas dataframe while Spark hides the implementation details of running it on many remote servers with varying loads and availability. 

Spark was developed at UC Berkeley and they ran several courses on edX to teach it to a wider audience (I completed a couple of those). Unfortunately they are now archived and it is unclear whether/when they will run again.

Astronomers haven't yet embraced Spark in a big way, but some LSST developers were interested enough to write and publish a FITS interface.

<a id='sql'></a>  

## Relational Databases and SQL

Big data with complex structures doesn't need to be handled entirely in Python, and often it would be silly to do so. Database management systems (DBMS's) have been around a long time, are highly optimized, and some very capable examples (PostgreSQL, MySQL/MariaDB) are free and open source. And they scale: postgres runs nicely on my laptop, and also powers searches of Hyper Suprime-Cam data and at least some Gaia and Hipparchos survey data (SDSS is older and uses SQL Server; LSST is a bit evasive about discussing technical details at this early stage).

### Do you need to learn SQL?

Of course! Beyond the simplest queries, this is the common language of data retrieval, regardless of what you want to do with it afterwards. It's fast, it's remarkably flexible, and when you have read-only access to someone else's database it can be terse and simple. Or as complicated as you want: there is almost no limit to how much you can pack into a single query.